## Proximal Policy Optimization 

В практической реализации существует два варианта реализации алгоритма PPO:
* выполняет обновление, ограниченное KL, как TRPO, но штрафует KL-расхождение в целевой функции вместо того, чтобы делать его жестким ограничением, и автоматически регулирует коэффициент штрафа в процессе обучения, чтобы он масштабировался соответствующим образом.
* не содержит в целевой функции члена KL-дивергенции и вообще не имеет ограничения. Вместо этого полагается на специализированный клиппинг 

<img src="https://spinningup.openai.com/en/latest/_images/math/e62a8971472597f4b014c2da064f636ffe365ba3.svg">

Спойлер: клиппинг - не самое главное в PPO, как это могло показаться на первый взгляд. Алгоритм PPO работает во многом и за счет небольших дополнительных улучшений. Подробнее: https://arxiv.org/pdf/2005.12729.pdf

### Задание 1: Заполните пропуски в алгоритме

In [5]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

print(COLAB)
if COLAB:
    !pip install "gymnasium[classic-control, atari, accept-rom-license]" --quiet
    !pip install piglet --quiet
    !pip install imageio_ffmpeg --quiet
    !pip install moviepy==1.0.3 --quiet
    !apt-get install swig -y
    !pip install gymnasium[box2d]

True
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 KB 35.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.5/67.5 KB 2.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 23 not upgraded.
Need to get 1,086 kB 

In [6]:

import torch
import torch.nn as nn
from torch.distributions import Categorical
import gymnasium as gym
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []

    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

### Сеть Actor-Critic

In [11]:
class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim):
        super().__init__()

        # actor: 2 hidden + output
        # self.action_layer = nn.Sequential(..., nn.Softmax(dim=-1))
        ####### Здесь ваш код ########
        self.action_layer = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, action_dim),
            nn.Softmax(-1)
        )
        ##############################

        # critic: 2 hidden + output
        # self.value_layer = nn.Sequential(...)
        ####### Здесь ваш код ########
        self.value_layer = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )
        ##############################

    def forward(self):
        action_probs = self.action_layer(state)
        state_value = self.value_layer(state)

    def act(self, state, memory):
        state = torch.from_numpy(state).float().to(device)
        
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        action = dist.sample()
        
        # сохраняем в память: state, action, log_prob(action)
        memory.states.append(state)
        memory.actions.append(action)
        memory.logprobs.append(dist.log_prob(action))


        return action.item()

    def evaluate(self, state, action):
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)

        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()

        state_value = self.value_layer(state)
        return action_logprobs, torch.squeeze(state_value), dist_entropy


### PPO policy

In [12]:
class PPO:
    def __init__(self, state_dim, action_dim, hidden_dim, lr, betas, gamma, K_epochs, eps_clip):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs

        self.policy = ActorCritic(state_dim, action_dim, hidden_dim).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr, betas=betas)
        self.policy_old = ActorCritic(state_dim, action_dim, hidden_dim).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())

        self.loss = nn.MSELoss()

    def update(self, memory):
        # Monte Carlo оценка вознаграждений:
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            # обнуляем накопленную награду, если попали в терминальное состояние
            if is_terminal:
                discounted_reward = 0
            # discounted_reward = 
            ####### Здесь ваш код ########
            discounted_reward += self.gamma * reward
            ##############################
            rewards.append(discounted_reward)

        rewards = torch.tensor(rewards[::-1], dtype=torch.float32).to(device)
        # выполните нормализацию вознаграждений (r - mean(r)) / std(r + 1e-5):
        ####### Здесь ваш код ########
        rewards = (rewards - rewards.mean()) / (torch.std(rewards) + 1e-6) 
        ##############################
        
        # конвертация list в tensor
        old_states = torch.stack(memory.states).to(device).detach()
        old_actions = torch.stack(memory.actions).to(device).detach()
        old_logprobs = torch.stack(memory.logprobs).to(device).detach()


        # оптимизация K epochs:
        for _ in range(self.K_epochs):
            # получаем logprobs, state_values, dist_entropy от старой стратегии:
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)


            # находим отношение стратегий (pi_theta / pi_theta_old), через logprobs и old_logprobs.detach():
            # ratios = 
            ####### Здесь ваш код ########
            ratios = torch.exp(logprobs - old_logprobs.detach())
            ##############################
    
            # считаем advantages
            # advantages = 
            ####### Здесь ваш код ########
            advantages = (rewards - state_values).detach()
            ##############################
            
            # Находим surrogate loss:
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1 - self.eps_clip, 1 + self.eps_clip) * advantages
            loss = -torch.min(surr1, surr2) + 0.5 * self.loss(state_values, rewards) - 0.01 * dist_entropy
            
            # делаем шаг градиента
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()

        # копируем веса
        self.policy_old.load_state_dict(self.policy.state_dict())

### Основной цикл

In [17]:
# env_name = "CartPole-v1"
env_name = "LunarLander-v2"
# env_name = "MountainCar-v0"

env = gym.make(env_name)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
render = False

# for LunarLander-v2 max reward is cut by 140 as stated in docs to dont relax no more
# source: https://www.gymlibrary.dev/environments/box2d/lunar_lander/
solved_reward = 110  # останавливаемся если avg_reward > solved_reward

log_interval = 20  # печатаем avg reward  в интервале 
max_episodes = 50000  # количество эпизодов обучения
max_timesteps = 500  # максимальное кол-во шагов в эпизоде
n_latent_var = 128  # кол-во переменных в скрытых слоях
update_timestep = 1000  # обновляем policy каждые n шагов
lr = 0.001 # learning rate
betas = (0.9, 0.999) # betas для adam optimizer
gamma = 0.99  # discount factor
K_epochs = 4  # количество эпох обноеления policy
eps_clip = 0.1  # clip параметр для PPO
random_seed = None

if random_seed:
    torch.manual_seed(random_seed)
    env.seed(random_seed)

memory = Memory()
ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip)

# переменные для логирования
running_reward = 0
avg_length = 0
timestep = 0

# цикл обучения
for i_episode in range(1, max_episodes + 1):
    state, _ = env.reset()
    for t in range(max_timesteps):
        timestep += 1

        # используем policy_old для выбора действия
        action = ppo.policy_old.act(state, memory)
        state, reward, terminated, truncated, _ = env.step(action)

        # сохраняем награды и флаги терминальных состояний:
        memory.rewards.append(reward)
        memory.is_terminals.append(terminated)

        # выполняем обновление
        if timestep % update_timestep == 0:
            ppo.update(memory)
            memory.clear_memory()
            timestep = 0

        running_reward += reward
        if render:
            env.render()
        if terminated or truncated:
            break

    avg_length += t

    # останавливаемся, если avg_reward > solved_reward
    if running_reward > (log_interval * solved_reward):
        print("########## Принято! ##########")
        torch.save(ppo.policy.state_dict(), './PPO_{}.pth'.format(env_name))
        break

    # логирование
    if i_episode % log_interval == 0:
        avg_length = int(avg_length / log_interval)
        running_reward = int((running_reward / log_interval))

        print('Episode {} \t avg length: {} \t reward: {}'.format(i_episode, avg_length, running_reward))
        running_reward = 0
        avg_length = 0

Episode 20 	 avg length: 88 	 reward: -196
Episode 40 	 avg length: 103 	 reward: -223
Episode 60 	 avg length: 86 	 reward: -167
Episode 80 	 avg length: 79 	 reward: -132
Episode 100 	 avg length: 95 	 reward: -168
Episode 120 	 avg length: 79 	 reward: -110
Episode 140 	 avg length: 83 	 reward: -135
Episode 160 	 avg length: 92 	 reward: -120
Episode 180 	 avg length: 88 	 reward: -106
Episode 200 	 avg length: 101 	 reward: -109
Episode 220 	 avg length: 96 	 reward: -102
Episode 240 	 avg length: 105 	 reward: -96
Episode 260 	 avg length: 99 	 reward: -65
Episode 280 	 avg length: 116 	 reward: -67
Episode 300 	 avg length: 96 	 reward: -63
Episode 320 	 avg length: 107 	 reward: -61
Episode 340 	 avg length: 132 	 reward: -72
Episode 360 	 avg length: 114 	 reward: -47
Episode 380 	 avg length: 138 	 reward: -34
Episode 400 	 avg length: 164 	 reward: -57
Episode 420 	 avg length: 153 	 reward: -25
Episode 440 	 avg length: 130 	 reward: -31
Episode 460 	 avg length: 132 	 rewa

### Задание 2: Попробуйте обучить алгоритм PPO, используя более сложную среду (например LunarLander-v2), для чего вам потребуется подобрать некотороые гиперпараметры.

**Работа со средой LunarLander в colab - установка pybox2d:**
1. Устанавливаем пакеты: 
```bash
!apt-get install swig -y
!pip install box2d-py
```
2. Перезапускаем runtime.